In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
# from sklearn_extra.cluster import KMedoids
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from statistics import mean
from numpy import isnan
from sklearn.impute import KNNImputer
import math
from scipy.spatial import distance
from fcmeans import FCM
from collections import Counter

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import f1_score

%matplotlib inline

In [2]:
# ,encoding= 'unicode_escape'
data = pd.read_csv('D:/TMU/data/patient.csv')
print(data.shape)
# data.head()
# data.describe()

(237, 22)


In [3]:
links_data = pd.read_csv('D:/TMU/data/links.csv')
print(links_data.shape)
# links_data.head()

(498, 3)


In [4]:
recoms_data = pd.read_csv('D:/TMU/data/recoms.csv')
print(recoms_data.shape)
# recoms_data.head()

(58, 2)


In [5]:
#drop rows with more than 11 NaN values
df = data
df.dropna(thresh=11, inplace=True)
print(df.shape)
np.sum(data.isna())

(230, 22)


id                                  0
weights                             9
heights                            20
SBP                                52
DBP                                61
age                                 4
sex                                 0
Family history of diabets          32
Family history of hypertension     19
chest pain                         16
Shortness of breath                19
arm muscle pain                    30
oily food                          21
sodium                             10
exercise                            8
smoker                              6
FBS                               100
chol                              151
heart rate                         79
heart rhythm                       77
BP type                             9
diabets                            46
dtype: int64

In [6]:
#drop columns with more than 50% NaN values
df.dropna(thresh=df.shape[0]*0.5,how='all',axis=1, inplace=True)

df = df.iloc[20:]

print(df.shape)
# np.sum(data.isna())

(210, 21)


In [7]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)
dataout = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
# df = dataout
# df.reset_index(drop = True, inplace=True)

df = df.sample(frac=1).reset_index(drop=True)

id                                109.50
weights                            18.00
heights                            14.00
SBP                                15.00
DBP                                10.00
age                                 1.00
sex                                 1.00
Family history of diabets           1.00
Family history of hypertension      1.00
chest pain                          1.00
Shortness of breath                 2.00
arm muscle pain                     2.00
oily food                           1.00
sodium                              1.00
exercise                            3.00
smoker                              2.25
FBS                                 0.00
heart rate                          1.00
heart rhythm                        0.00
BP type                             1.00
diabets                             0.00
dtype: float64


In [8]:
df_with_id = df

In [9]:
df

,id,weights,heights,SBP,DBP,age,sex,Family history of diabets,Family history of hypertension,chest pain,...,arm muscle pain,oily food,sodium,exercise,smoker,FBS,heart rate,heart rhythm,BP type,diabets
0,44,74.0,180.0,125.0,80.0,3.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,NaN,3.0,0.0,0.0,0.0
1,53,81.0,175.0,140.0,80.0,4.0,1.0,0.0,0.0,0.0,...,0.0,0.0,2.0,1.0,0.0,NaN,3.0,NaN,1.0,NaN
2,151,74.0,172.0,120.0,80.0,3.0,1.0,0.0,1.0,2.0,...,2.0,0.0,1.0,4.0,3.0,0.0,NaN,0.0,1.0,0.0
3,223,100.0,177.0,130.0,80.0,3.0,1.0,1.0,1.0,0.0,...,2.0,1.0,1.0,4.0,5.0,1.0,NaN,NaN,1.0,1.0
4,67,66.0,157.0,120.0,60.0,2.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,1.0,3.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,208,62.0,151.0,130.0,70.0,3.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0
206,100,70.0,165.0,120.0,80.0,2.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,NaN,3.0,0.0,1.0,0.0
207,24,93.0,174.0,NaN,NaN,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,4.0,0.0,NaN,NaN,1.0,0.0
208,90,81.0,167.0,120.0,70.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,4.0,NaN,3.0,0.0,0.0,0.0


In [10]:
# #fill missing data in each categorical column with its mode
# df['age'] = df['age'].fillna(df['age'].mode()[0])
# df['sex'] = df['sex'].fillna(df['sex'].mode()[0])
# df['Family history of diabets'] = df['Family history of diabets'].fillna(df['Family history of diabets'].mode()[0])
# df['Family history of hypertension'] = df['Family history of hypertension'].fillna(df['Family history of hypertension'].mode()[0])
# df['chest pain'] = df['chest pain'].fillna(df['chest pain'].mode()[0])
# df['Shortness of breath'] = df['Shortness of breath'].fillna(df['Shortness of breath'].mode()[0])
# df['arm muscle pain'] = df['arm muscle pain'].fillna(df['arm muscle pain'].mode()[0])
# df['oily food'] = df['oily food'].fillna(df['oily food'].mode()[0])
# df['sodium'] = df['sodium'].fillna(df['sodium'].mode()[0])
# df['exercise'] = df['exercise'].fillna(df['exercise'].mode()[0])
# df['smoker'] = df['smoker'].fillna(df['smoker'].mode()[0])
# df['heart rate'] = df['heart rate'].fillna(df['heart rate'].mode()[0])
# df['heart rhythm'] = df['heart rhythm'].fillna(df['heart rhythm'].mode()[0])
# df['BP type'] = df['BP type'].fillna(df['BP type'].mode()[0])
# df['diabets'] = df['diabets'].fillna(df['diabets'].mode()[0])
# # np.sum(df.isna())


# #fill missing data in weight and height columns with its median
# df['weights'] = df['weights'].fillna(df['weights'].median())
# df['heights'] = df['heights'].fillna(df['heights'].median())


# #fill missing data in DBP and SBP columns with its median
# df['DBP'] = df['DBP'].fillna(df['DBP'].median())
# df['SBP'] = df['SBP'].fillna(df['SBP'].median())
# np.sum(df.isna())


In [11]:
# create Blood Pressure type binding using SBP & DBP

bloodPressureArray =[]
for row in df.index:
#     print(df[df.index==row]['SBP'])
    SBP =df[df.index==row]['SBP'].values
    DBP =df[df.index==row]['DBP'].values
    if(np.isnan(SBP) or np.isnan(DBP)):
        bloodPressureArray.append("NaN")
    elif(SBP<130 and DBP<85):
        bloodPressureArray.append(0)
#     elif(120<=SBP<140 or 80<=DBP<90):
#         bloodPressureArray.append(1)  
#     elif(140<=SBP<160 or 90<=DBP<99):
#         bloodPressureArray.append(2) 
    else:
        bloodPressureArray.append(1) 
        

df.insert(1, 'BPClass', bloodPressureArray)
del df['SBP']
del df['DBP']
# df

In [12]:
# create Blood Pressure type binding using SBP & DBP
BMIarray =[]
for row in df.index:
#     print(df[df.index==row]['SBP'])
    H =df[df.index==row]['heights'].values
    W =df[df.index==row]['weights'].values
    BMI = W / (H/100)**2
    if(np.isnan(W) or np.isnan(H)):
        BMIarray.append("NaN")
    elif BMI < 18.5:
        BMIarray.append(0)
    elif 18.5 <= BMI < 25:
        BMIarray.append(1)
    elif 25 <= BMI < 30:
        BMIarray.append(2)
    elif 30 <= BMI < 40:
        BMIarray.append(3)
    else:
        BMIarray.append(4)

df.insert(1, 'BMI', BMIarray)
del df['heights']
del df['weights']
del df['FBS']



In [13]:
#fill missing values with running a model --- age

#1-- Make missing records as our Testing data and non-missing records as our Training data.
# mask = df["age"].isnull()
# test_age = df[mask]
# train_age = df[~mask]


# X= data.drop(['BP type'],axis=1)
X_impute= df.drop(['id'],axis=1)

# define imputer
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
# fit on the dataset
imputer.fit(X_impute)
# transform the dataset
Xtrans = imputer.transform(X_impute)
# print total missing
print('Missing: %d' % sum(isnan(Xtrans).flatten()))
df=pd.DataFrame(np.round(Xtrans) ,columns = X_impute.columns)

Missing: 0


In [14]:
#which similarity metrics to use
similarity_selection = 'cosine'

#number of clusters
k = 8

#test & train sampling
train_size = int(0.9 * len(df))

df_train = df.sample(n=train_size, replace=False)
df_test = pd.concat([df, df_train]).drop_duplicates(keep=False)
# df_test = df.append(df_train).drop_duplicates(keep=False)
X_test = df_test
print(df.shape)
print(df_train.shape)
print(df_test.shape)

(210, 17)
(189, 17)
(21, 17)


# Models::

In [15]:

kmeans_model = KMeans(n_clusters = k, random_state=1)
kmeans_model.fit(df_train)
labelsList = kmeans_model.labels_
metrics.silhouette_score(df_train, labelsList, metric='euclidean')
y_kmeans = kmeans_model.predict(df_train)
predict = kmeans_model.fit_predict(df_train)

In [16]:
labels = pd.DataFrame(kmeans_model.labels_)
# labeledforplots = pd.concat((df,labels),axis=1)
labeled_df = df_with_id.join(labels)
labeled_df = labeled_df.rename({0:'labels'},axis=1)

# sns.pairplot(labeled_df,hue='labels')

In [17]:
recoms_list_per_cluster=[]
for i in range(0,k):
    data_in_cluster = pd.DataFrame(labeled_df[labeled_df.labels == i])
    recoms = []
    listt =[]
    for row in data_in_cluster.values:
        recoms = links_data[links_data.patientID==row[0]]['recomID'].values.tolist()
        for recoms in recoms:
            listt.append(recoms)
    list_df = pd.DataFrame(listt)
    unique_list =np.unique(np.array(list_df)) 
    recoms_list_per_cluster.append(unique_list)

# recoms_list_per_cluster = pd.DataFrame(recoms_list_per_cluster)
# recoms_list_per_cluster

In [18]:
# sns.lmplot(x='heights',y='weights',data=labeled_df,hue='labels',fit_reg=False)

In [19]:
#add cluster label to each row in df data
#labeled_df

print(df.shape)
print(df_with_id.shape)
print(labels.shape)
print(labeled_df.shape)
# print(recoms_list_per_cluster.shape)


(210, 17)
(210, 18)
(189, 1)
(210, 19)


In [20]:
#cosine_similarity 
cosineSim = cosine_similarity(X_test, df_train)
cosine_similarity_df = pd.DataFrame(cosineSim)
print(cosine_similarity_df.shape)


(21, 189)


In [21]:
#Jaccard Similarity  

Jaccard_Similarity = []
# print(X.shape)
# print(X_test.index)
# print(df.index)
for i in X_test.index:
#     print(i)
    test_item = df.iloc[i]
    similarity_list = []
    
    for j in df_train.index:
#         print(int(j))
        train_item = df.iloc[int(j)]
        intersection = len(list(set(test_item).intersection(train_item)))
        union = (len(test_item) + len(train_item)) - intersection
        result = float(intersection) / union
        similarity_list.append(result)
    Jaccard_Similarity.append(similarity_list)
Jaccard_Similarity_df = pd.DataFrame(Jaccard_Similarity)


In [22]:
#set similarity_selection before model start
if(similarity_selection == "cosine"):
    similarity_df = cosine_similarity_df
elif(similarity_selection == "jaccard"):
    similarity_df = Jaccard_Similarity_df

In [23]:
# get top three sorted for all rows
    
top_three_sorted = []

for row in similarity_df.index:
    newitem = similarity_df.iloc[row].sort_values(ascending=False)[1:4]
    top_three_sorted.append(newitem)
top_three_sorted_df =pd.DataFrame(np.array(top_three_sorted))

# top_three_sorted_df

In [24]:
# get top three sorted with data such as cluster id and patient's id for all rows

user_id = []
similar_id_1 = []
value_1 = []
cluster_id1 = []
similar_id_2 = []
value_2 = []
cluster_id2 = []
similar_id_3 = []
value_3 = []
cluster_id3 = []

top_three_similar = {}

for row in top_three_sorted_df.index:
    
#     user_id.append(index1)
    
    index1 = similarity_df.iloc[row].tolist().index(top_three_sorted_df.iloc[row][0])
#     print(row)
#     print()
#     print(similarity_df.iloc[row].tolist())
#     print()
#     print(top_three_sorted_df.iloc[row][0])
    similar_id_1.append(index1)
    value_1.append(top_three_sorted_df.iloc[row][0])
    cluster_id1.append(labels.iloc[index1][0])

    #     print(index1)
    #     print(top_three_sorted_df.iloc[row][0])
    #     print(labels.iloc[index1])
    
    index2 = similarity_df.iloc[row].tolist().index(top_three_sorted_df.iloc[row][1])
    similar_id_2.append(index2)
    value_2.append(top_three_sorted_df.iloc[row][1])
    cluster_id2.append(labels.iloc[index2][0])

    index3 = similarity_df.iloc[row].tolist().index(top_three_sorted_df.iloc[row][2])
    similar_id_3.append(index3)
    value_3.append(top_three_sorted_df.iloc[row][2])
    cluster_id3.append(labels.iloc[index3][0])
    
#     if(labels.iloc[index1][0] != labels.iloc[index2][0] or labels.iloc[index2][0] != labels.iloc[index3][0] or labels.iloc[index1][0] != labels.iloc[index3][0]):
#         print("id:",row,labels.iloc[index1][0], labels.iloc[index2][0], labels.iloc[index3][0],'/n')
        
top_three_similar = {
    'similar_id_1': similar_id_1,
    'value_1': value_1,
    'cluster_id1':cluster_id1,

    'similar_id_2': similar_id_2,
    'value_2': value_2,
    'cluster_id2':cluster_id2,

    'similar_id_3': similar_id_3,
    'value_3': value_3,
    'cluster_id3':cluster_id3,
    }

top_three_similar_df = pd.DataFrame(top_three_similar)
top_three_similar_df

,similar_id_1,value_1,cluster_id1,similar_id_2,value_2,cluster_id2,similar_id_3,value_3,cluster_id3
0,51,0.948122,1,120,0.945594,1,88,0.943133,4
1,188,0.913664,4,2,0.913547,4,50,0.906293,4
2,183,0.927961,4,181,0.906693,1,171,0.903211,1
3,51,0.933799,1,111,0.932812,1,120,0.932505,1
4,85,0.956681,1,133,0.953463,1,184,0.940939,1
5,167,0.886186,1,100,0.880705,1,150,0.875190,6
6,88,0.971483,4,45,0.958315,4,65,0.947331,1
7,7,0.947373,3,176,0.938315,5,142,0.929070,3
8,170,0.932294,2,4,0.925820,2,107,0.923936,2
9,11,0.953591,3,7,0.945992,3,61,0.938035,3


In [25]:
def calculate_common(a):
    common_id = []
    for i in a:
        if a.count(i) >= 2:
            common_id.append(i)
    return common_id

In [26]:

accuracy_list = []
precision_list = []
recall_list = []
mae_list = []
counter = 0

for i in top_three_similar_df.index:
    
    row = top_three_similar_df.iloc[i]
    new_recoms_intersection_set = set()
    real_recoms_intersection_set = set()

    real_recoms = links_data[links_data.patientID==int(i)]['recomID'].values.tolist()
    real_recoms_intersection_set = set(real_recoms)
    
    cluster3_recoms = recoms_list_per_cluster[int(row.cluster_id3)]
    new_recoms_intersection_set = set(cluster3_recoms) 
    
    cluster2_recoms = recoms_list_per_cluster[int(row.cluster_id2)]
    new_recoms_intersection_set = new_recoms_intersection_set.intersection(cluster2_recoms)
    
    cluster1_recoms = recoms_list_per_cluster[int(row.cluster_id1)]
    new_recoms_intersection_set = new_recoms_intersection_set.intersection(cluster1_recoms)
    
    print(i)
    print("new_recoms: ", new_recoms_intersection_set)
    print("real_recoms: ", real_recoms_intersection_set)
  
    
    if len(real_recoms_intersection_set) != 0:
        counter = counter + 1
        compare_list = real_recoms_intersection_set.intersection(new_recoms_intersection_set)
        
        accuracy = (len(compare_list) / len(real_recoms_intersection_set))*100
        accuracy_list.append(accuracy)

        precision = 0
        if len(new_recoms_intersection_set) != 0:
            precision = (len(compare_list) / len(new_recoms_intersection_set))*100
        precision_list.append(precision)
        
        recall = (len(compare_list) / len(real_recoms_intersection_set))*100
        recall_list.append(recall)
        
        #compute MAE:::
        realArray = []
        predArray = []
        subtractArray = []
        for i in range(len(recoms_data)):
            ID = recoms_data.iloc[i].id
            
            if ID in real_recoms_intersection_set:
                realArray.append(1)
            else:
                realArray.append(0)
                
            if ID in new_recoms_intersection_set:
                predArray.append(1)
            else:
                predArray.append(0)
                
            subtractArray.append(abs(predArray[i] - realArray[i]))
            
        mae_list.append(sum(subtractArray)/len(recoms_data)*100)
        
        
    
        print("compare_list: ", compare_list)
#         print("accuracy: ", accuracy)
        print("precision: ", precision)
        print("recall: ", recall)
      
        
    print()

print("num ofpatients with recoms:", counter)
# print(accuracy_list)





0
new_recoms:  {35, 67, 18, 51, 20, 21, 22, 55, 63, 25, 59, 61, 30, 57}
real_recoms:  set()

1
new_recoms:  {18, 19, 20, 21, 22, 25, 30, 33, 34, 35, 36, 38, 40, 41, 46, 48, 49, 51, 53, 55, 57, 59, 61, 63, 67, 68}
real_recoms:  set()

2
new_recoms:  {35, 67, 18, 51, 20, 21, 22, 55, 63, 25, 59, 61, 30, 57}
real_recoms:  set()

3
new_recoms:  {65, 2, 35, 67, 37, 8, 9, 18, 51, 20, 21, 22, 55, 63, 25, 59, 57, 61, 30, 31}
real_recoms:  {0}
compare_list:  set()
precision:  0.0
recall:  0.0

4
new_recoms:  {65, 2, 35, 67, 37, 8, 9, 18, 51, 20, 21, 22, 55, 63, 25, 59, 57, 61, 30, 31}
real_recoms:  {2}
compare_list:  {2}
precision:  5.0
recall:  100.0

5
new_recoms:  {35, 67, 8, 9, 51, 20, 21, 22, 55, 57, 59, 61, 30}
real_recoms:  set()

6
new_recoms:  {35, 67, 18, 51, 20, 21, 22, 55, 63, 25, 59, 61, 30, 57}
real_recoms:  {1}
compare_list:  set()
precision:  0.0
recall:  0.0

7
new_recoms:  {35, 67, 55, 57, 59, 61}
real_recoms:  set()

8
new_recoms:  {7, 11, 18, 19, 22, 25, 26, 29, 30, 35, 36, 3

In [27]:
#calculate accuracy
# avrg_accuracy = mean(accuracy_list) 
# print("accuracy mean: ", avrg_accuracy)

#calculate precision
avrg_precision = mean(precision_list) 
print("precision mean: ", avrg_precision)

#calculate recall
avrg_recall = mean(recall_list) 
print("recall mean: ", avrg_recall)

print("fmeasure: ", (2*(avrg_recall*avrg_precision)/(avrg_recall+avrg_precision)))

#calculate MAE
avrg_mae = mean(mae_list) 
print("MAE mean: ", avrg_mae)

precision mean:  4.562728937728937
recall mean:  30.625
fmeasure:  7.942176317501627
MAE mean:  32.327586206896555
